In [ ]:
from flask import Flask, render_template, request, redirect, url_for, session, jsonify
def user_session():
    if request.method == 'POST':
        session['name'] = request.form['name']
        return redirect(url_for('user_session'))
    return f"Hello, {session.get('name', 'Guest')}!"

@app.route('/upload', methods=['POST'])
def upload():
    file = request.files['file']
    if file:
        file_path = os.path.join(app.config['UPLOAD_FOLDER'], file.filename)
        file.save(file_path)
        return "File uploaded successfully!"
    return "Upload a file."

@app.route('/items', methods=['GET', 'POST'])
def manage_items():
    if request.method == 'POST':
        name = request.form['name']
        item = Item(name=name)
        db.session.add(item)
        db.session.commit()
    items = Item.query.all()
    return render_template('items.html', items=items)

@app.route('/register', methods=['POST'])
def register():
    username = request.form['username']
    password = request.form['password']
    user = User(username=username, password=password)
    db.session.add(user)
    db.session.commit()
    return "User registered!"

@app.route('/login', methods=['POST'])
def login():
    username = request.form['username']
    user = User.query.filter_by(username=username).first()
    if user:
        login_user(user)
        return redirect(url_for('dashboard'))
    return "Invalid login!"

@app.route('/dashboard')
@login_required
def dashboard():
    return f"Welcome {current_user.username}!"

@app.route('/logout')
def logout():
    logout_user()
    return redirect(url_for('home'))

@app.route('/api/items', methods=['GET'])
def api_items():
    items = Item.query.all()
    return jsonify([{ 'id': i.id, 'name': i.name } for i in items])

@app.errorhandler(404)
def not_found_error(error):
    return "Page not found", 404

@app.errorhandler(500)
def internal_error(error):
    return "Internal server error", 500

@socketio.on('message')
def handle_message(msg):
    send(msg, broadcast=True)

if __name__ == '__main__':
    db.create_all()
    socketio.run(app, debug=True)
